In [ ]:
# https://towardsdatascience.com/how-to-use-the-staple-algorithm-to-combine-multiple-image-segmentations-ce91ebeb451e
# packages
import nibabel as nib # https://nipy.org/nibabel/
import SimpleITK as sitk # https://simpleitk.org/
from matplotlib import pyplot as plt
from meidic_vtach_utils.run_on_recommended_cuda import get_cuda_environ_vars as get_vars
from collections import OrderedDict
import torch
import numpy as np

In [70]:
reg_state = "acummulate_deeds_FT2_MT1"

staple_filter = sitk.STAPLEImageFilter()
# sitk.ProcessObject.SetGlobalDefaultDebugOff()
FOREGROUND = 1.0
weight_data = {}
staple_filter.SetForegroundValue(FOREGROUND)

if reg_state == "acummulate_deeds_FT2_MT1":
    bare_data = torch.load("/share/data_supergrover1/weihsbach/shared_data/important_data_artifacts/curriculum_deeplab/20220114_crossmoda_multiple_registrations/crossmoda_deeds_registered.pth")
    for fixed_id, moving_dict in bare_data.items():
        moving_data = []
        # loaded_identifier = []
        selected_moving_ids = []
        sorted_moving_dict = OrderedDict(moving_dict)
        for idx_mov, (moving_id, moving_sample) in enumerate(sorted_moving_dict.items()):
            # Only use every third warped sample
            if idx_mov % 3 == 0:
                moving_data.append(moving_sample['warped_label'].cpu())
                selected_moving_ids.append(moving_id)
                # loaded_identifier.append(f"{fixed_id}:m{moving_id}")
        
        sitk_moving_data = [sitk.GetImageFromArray(reg_seg.to_dense().numpy().astype(np.int16)) for reg_seg in moving_data]
    
        staple_out = staple_filter.Execute(sitk_moving_data)
        # staple_out = sitk.STAPLE(sitk_moving_data, FOREGROUND)
        consensus = sitk.GetArrayFromImage(staple_out)
        
        specitivity = staple_filter.GetSpecificity()
        sensitivity = staple_filter.GetSensitivity()
        f_weight_dict = weight_data.get(fixed_id, {})
        staple_consensus = sitk.GetArrayFromImage(staple_out)

        for moving_id, sens, spec in zip(selected_moving_ids, sensitivity, specitivity):
            f_weight_dict[moving_id] = dict(sensitivity=sens, specitivity=spec)
        weight_data[fixed_id] = f_weight_dict
else:
    raise ValueError()

Debug: STAPLEImageFilter (0x55ec92bd6990): Executing ITK filter:
STAPLEImageFilter (0x55ec94a65dd0)
  RTTI typeinfo:   itk::STAPLEImageFilter<itk::Image<short, 3u>, itk::Image<double, 3u> >
  Reference Count: 1
  Modified Time: 15449
  Debug: Off
  Object Name: 
  Observers: 
    DeleteEvent(FunctionCommand)
  Inputs: 
    Primary: (0x55ec929a6ce0) *
    _1: (0x55ec92edc400)
    _2: (0x55ec94492e00)
    _3: (0x55ec9484b5c0)
    _4: (0x55ec9287a120)
    _5: (0x55ec945a2e90)
    _6: (0x55ec49dedc80)
    _7: (0x55ec947f16d0)
    _8: (0x55ec929af650)
    _9: (0x55ec9298a9f0)
  Indexed Inputs: 
    0: Primary (0x55ec929a6ce0)
    1: _1 (0x55ec92edc400)
    2: _2 (0x55ec94492e00)
    3: _3 (0x55ec9484b5c0)
    4: _4 (0x55ec9287a120)
    5: _5 (0x55ec945a2e90)
    6: _6 (0x55ec49dedc80)
    7: _7 (0x55ec947f16d0)
    8: _8 (0x55ec929af650)
    9: _9 (0x55ec9298a9f0)
  Required Input Names: Primary
  NumberOfRequiredInputs: 1
  Outputs: 
    Primary: (0x55ec92e3c550)
  Indexed Outputs: 
    0:

In [ ]:
print(weight_data.keys())
print(weight_data['108r'].keys())